In [14]:
import os
import shutil

# Define current root folder (your current folder structure with train/test inside category folders)
current_root = 'images'

# Define the new structure's root directory
new_root = 'organized_data'

# Define categories (adjust according to your categories)
categories = ['Coast', 'Forest', 'Highway', 'Kitchen', 'Mountain', 
              'Office', 'Store', 'Street', 'Suburb']

# Create the new folder structure for train and validation
for category in categories:
    os.makedirs(os.path.join(new_root, 'train', category), exist_ok=True)
    os.makedirs(os.path.join(new_root, 'validation', category), exist_ok=True)

# Move files from the old structure to the new structure
for category in categories:
    # Define source paths
    old_train_dir = os.path.join(current_root, category, 'train')
    old_test_dir = os.path.join(current_root, category, 'test')
    
    # Define destination paths
    new_train_dir = os.path.join(new_root, 'train', category)
    new_validation_dir = os.path.join(new_root, 'validation', category)
    
    # Move the training images to the new train directory
    if os.path.exists(old_train_dir):
        for filename in os.listdir(old_train_dir):
            if filename.endswith('.jpg'):  # Assuming all files are images
                shutil.move(os.path.join(old_train_dir, filename), os.path.join(new_train_dir, filename))
    
    # Move the test images to the new validation directory
    if os.path.exists(old_test_dir):
        for filename in os.listdir(old_test_dir):
            if filename.endswith('.jpg'):
                shutil.move(os.path.join(old_test_dir, filename), os.path.join(new_validation_dir, filename))

print("Files have been moved to the new train and validation folders.")

Files have been moved to the new train and validation folders.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up directories and image size
train_dir = 'organized_data/train'
test_dir = 'organized_data/validation'
img_size = 224
batch_size = 64  # Keep the batch size smaller for the smaller dataset

# Define the image data generators for loading the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# VGG16-inspired model
model = models.Sequential()

# First convolutional stage (2 Conv layers + MaxPooling)
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second convolutional stage (2 Conv layers + MaxPooling)
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Third convolutional stage (3 Conv layers + MaxPooling)
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layer
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))  # Reduced from VGG's 4096

# Output layer with softmax for 9 categories
model.add(layers.Dense(9, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.4f}')

Found 902 images belonging to 9 classes.
Found 1698 images belonging to 9 classes.


2024-10-18 09:01:03.253530: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-18 09:01:03.257510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-18 09:01:03.257659: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-18 09:01:03.257950: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/20


2024-10-18 09:01:04.914716: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2024-10-18 09:01:05.292245: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-18 09:01:05.292511: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-18 09:01:05.292528: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2024-10-18 09:01:05.292863: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-18 09:01:05.292908: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-10-18 09:01:06.057720: W tensorflow/core/common_runtime/bf

14/14 [==============================] - 17s 798ms/step - loss: 2.1712 - accuracy: 0.1241 - val_loss: 2.0528 - val_accuracy: 0.2356
Epoch 2/20
14/14 [==============================] - 8s 555ms/step - loss: 1.8449 - accuracy: 0.3377 - val_loss: 1.7398 - val_accuracy: 0.4081
Epoch 3/20
14/14 [==============================] - 8s 567ms/step - loss: 1.4643 - accuracy: 0.5072 - val_loss: 1.4696 - val_accuracy: 0.4976
Epoch 4/20
14/14 [==============================] - 8s 562ms/step - loss: 1.1730 - accuracy: 0.6295 - val_loss: 1.4108 - val_accuracy: 0.5216
Epoch 5/20
14/14 [==============================] - 8s 547ms/step - loss: 0.9637 - accuracy: 0.6814 - val_loss: 1.3937 - val_accuracy: 0.5553
Epoch 6/20
14/14 [==============================] - 8s 570ms/step - loss: 0.9056 - accuracy: 0.7143 - val_loss: 1.1566 - val_accuracy: 0.6022
Epoch 7/20
14/14 [==============================] - 8s 575ms/step - loss: 0.6947 - accuracy: 0.7697 - val_loss: 1.1252 - val_accuracy: 0.6184
Epoch 8/20
14/14